In [26]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

from __future__ import print_function
from __future__ import division
from bayes_opt import BayesianOptimization

from sklearn.model_selection import KFold

In [2]:
# df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df=pd.read_csv('./ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

In [3]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d
df_dict=split_df(df);

In [4]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [5]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [6]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [7]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=df_dict[sku]['Sales'].rolling( window=7).mean().shift(1)

In [8]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [9]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [10]:
len(X[1])

935

In [11]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [12]:
cv=2

def RF(n_est,max_d,max_f):
    RF=RandomForestRegressor( n_estimators=int(n_est),max_depth=int(max_d),max_features=int(max_f), criterion='mae')
    rf_models=[];
    for sku in list(df_dict.keys()):
        rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0))
    return np.array(rf_models).mean()


def GB(n_est,max_d,max_f):
    GB=ensemble.GradientBoostingRegressor(n_estimators=int(n_est),max_depth=int(max_d),max_features=int(max_f), loss='lad');
    gb_models=[];
    for sku in list(df_dict.keys()):
        gb_models.append(cross_val_score(GB,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0))
    return np.array(gb_models).mean()   
    
def XG(n_est,lr,max_d,g):
    XG=xgb.XGBRegressor(n_estimators= int(n_est),learning_rate =lr, max_depth=int(max_d),gamma=int(g), eval_metric='mae')
    xg_models=[]
    for sku in list(df_dict.keys()):
        xg_models.append(cross_val_score(XG,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) ) 
    return np.array(xg_models).mean()
    

In [13]:
n_iter=2
gp_params = {"alpha": 1e-5}
X[1]

array([[ 0.4526554 ,  1.26667155,  2.00670918, ...,  0.78699665,
        -1.95640391, -0.24957802],
       [ 0.4526554 ,  1.26667155,  2.00670918, ...,  0.18882561,
         0.78771656,  1.02261947],
       [ 0.4526554 ,  1.26667155,  2.00670918, ...,  0.26788562,
         0.19075871,  1.00916792],
       ...,
       [ 0.4526554 ,  1.26667155,  2.00670918, ..., -0.08863028,
        -0.23748287, -0.27237726],
       [ 0.4526554 ,  1.26667155,  2.00670918, ..., -0.11995595,
        -0.08613446,  0.0281167 ],
       [ 0.4526554 ,  1.26667155,  2.00670918, ...,  0.20722322,
        -0.11739659,  0.31333517]])

In [25]:
### HYPERBAND STAGE 1


data = { 'x_train': X[1], 'y_train': y[1], 'x_test': X[1], 'y_test': y[1] }

import hyperopt as hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.stochastic import sample
from sklearn.metrics import roc_auc_score as AUC, log_loss, accuracy_score as accuracy
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from pprint import pprint

def handle_integers(params):
    new_params = {}
    for k, v in list(params.items()):
        if type(v) == float and int(v) == v:
            new_params[k] = int(v)
        else:
            new_params[k] = v

    return new_params

# "clf", even though it's a regressor
def train_and_eval_sklearn_regressor(clf, data, scaler=None):
    y_train = data['y_train']
    y_test = data['y_test']

    if scaler:
        scaler = eval("{}()".format(scaler))
        x_train = scaler.fit_transform(data['x_train'].astype(float))
        x_test = scaler.transform(data['x_test'].astype(float))
    else:
        x_train = data['x_train']
        x_test = data['x_test']

    clf.fit(x_train, y_train)
    p = clf.predict(x_train)

    mse = MSE(y_train, p)
    rmse = sqrt(mse)
    mae = MAE(y_train, p)

    print("\n# training | RMSE: {:.4f}, MAE: {:.4f}".format(rmse, mae))


    p = clf.predict(x_test)

    mse = MSE(y_test, p)
    rmse = sqrt(mse)
    mae = MAE(y_test, p)

    print("# testing  | RMSE: {:.4f}, MAE: {:.4f}".format(rmse, mae))
    return {'loss': rmse, 'rmse': rmse, 'mae': mae}


NameError: name 'sklearn' is not defined

In [47]:
kf = KFold(n_splits=3)
# kf.get_n_splits(X[1])

for train_index, test_index in kf.split(X[1]):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[1][train_index], X[1][test_index]
    y_train, y_test = y[1][train_index], y[1][test_index]
    
    
# kf = KFold(n_splits=2)

# train_index, text_index = kf.split(X[1])
# # X[1][train_index,:]
# train_index
# test_index

TRAIN: [312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329
 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347
 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365
 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383
 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401
 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419
 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437
 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455
 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473
 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491
 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509
 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527
 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545
 546 547 548 549 550 551 552 553 554 555 556

In [20]:
### HYPERBAND STAGE 2

trees_per_iteration = 5

space = {
# 'criterion': hp.choice( 'c', ( 'mse', 'mae' )),		# sklearn 0.18
# 'bootstrap': hp.choice( 'b', ( True, False )),
'max_depth': hp.quniform( 'md', 2, 10, 1 ),
# 'max_features': hp.choice( 'mf', ( 'sqrt', 'log2', None ))
'max_features': hp.choice( 'mf', ( 5, 10))
# 'min_samples_split': hp.quniform( 'msp', 2, 20, 1 ),
# 'min_samples_leaf': hp.quniform( 'msl', 1, 10, 1 ),
}

def get_params():
    params = sample(space)
    return handle_integers(params)



###  To get this to work for 4 model families, maybe we just need 4 different try_params?  try_params_RF, try_params_XGB, etc???

def try_params(n_iterations, params, data=data):
    n_estimators = int(round(n_iterations * trees_per_iteration))
    print("n_estimators:", n_estimators)
    pprint(params)

    clf = RF(n_estimators=n_estimators, verbose=0, n_jobs=-1, **params)

    return train_and_eval_sklearn_regressor(clf, data)


In [23]:
### HYPERBAND STAGE 3

from math import log, ceil, sqrt
from time import time, ctime


class Hyperband:
    def __init__(self, get_params_function, try_params_function, max_iter=5, eta=3):
        self.get_params = get_params_function
        self.try_params = try_params_function

        self.max_iter = max_iter  # maximum iterations per configuration
        self.eta = eta # defines configuration downsampling rate (default = 3)

        self.logeta = lambda x: log(x) / log(self.eta)
        self.s_max = int(self.logeta(self.max_iter))
        self.B = (self.s_max + 1) * self.max_iter

        self.results = []  # list of dicts
        self.counter = 0
        self.best_loss = np.inf
        self.best_counter = -1

    # can be called multiple times
    def run(self, skip_last=0, dry_run=False):

        for s in reversed(list(range(self.s_max + 1))):

            # initial number of configurations
            n = int(ceil(self.B / self.max_iter / (s + 1) * self.eta ** s))

            # initial number of iterations per config
            r = self.max_iter * self.eta ** (-s)

            # n random configurations
            T = [self.get_params() for i in range(n)]

            for i in range((s + 1) - int(skip_last)):  # changed from s + 1

                # Run each of the n configs for <iterations>
                # and keep best (n_configs / eta) configurations

                n_configs = n * self.eta ** (-i)
                n_iterations = r * self.eta ** (i)

                print("\n*** {} configurations x {:.1f} iterations each".format(
                    n_configs, n_iterations))

                val_losses = []
                early_stops = []

                for t in T:

                    self.counter += 1
                    print("\n{} | {} | lowest loss so far: {:.4f} (run {})\n".format(
                        self.counter, ctime(), self.best_loss, self.best_counter))

                    start_time = time()

                    if dry_run:
                        result = {'loss': random(), 'log_loss': random(), 'auc': random()}
                    else:
                        result = self.try_params(n_iterations, t)  # <---

                    assert (type(result) == dict)
                    assert ('loss' in result)

                    seconds = int(round(time() - start_time))
                    print("\n{} seconds.".format(seconds))

                    loss = result['loss']
                    val_losses.append(loss)

                    early_stop = result.get('early_stop', False)
                    early_stops.append(early_stop)

                    # keeping track of the best result so far (for display only)
                    # could do it be checking results each time, but hey
                    if loss < self.best_loss:
                        self.best_loss = loss
                        self.best_counter = self.counter

                    result['counter'] = self.counter
                    result['seconds'] = seconds
                    result['params'] = t
                    result['iterations'] = n_iterations

                    self.results.append(result)

                # select a number of best configurations for the next loop
                # filter out early stops, if any
                indices = np.argsort(val_losses)
                T = [T[i] for i in indices if not early_stops[i]]
                T = T[0:int(n_configs / self.eta)]

        return self.results

In [24]:
# We need to pass get_params a data argument. 
# Redefine RF just to get this to work

from sklearn.ensemble import RandomForestRegressor as RF

hb = Hyperband(get_params, try_params)

results = hb.run(skip_last=1)


*** 3 configurations x 1.7 iterations each

1 | Tue Jul 10 11:26:44 2018 | lowest loss so far: inf (run -1)

n_estimators: 8
{'max_depth': 9, 'max_features': 5}

# training | RMSE: 405.6462, MAE: 280.8284
# testing  | RMSE: 405.6462, MAE: 280.8284

0 seconds.

2 | Tue Jul 10 11:26:44 2018 | lowest loss so far: 405.6462 (run 1)

n_estimators: 8
{'max_depth': 8, 'max_features': 10}

# training | RMSE: 383.8099, MAE: 267.2993
# testing  | RMSE: 383.8099, MAE: 267.2993

0 seconds.

3 | Tue Jul 10 11:26:45 2018 | lowest loss so far: 383.8099 (run 2)

n_estimators: 8
{'max_depth': 6, 'max_features': 10}

# training | RMSE: 477.7272, MAE: 335.0926
# testing  | RMSE: 477.7272, MAE: 335.0926

0 seconds.


In [49]:
from hyperopt import fmin, tpe, hp

best = fmin(fn=lambda x: x ** 2,

    space=hp.uniform('x', -10, 10),

    algo=tpe.suggest,

    max_evals=100)

print(best)

TypeError: 'generator' object is not subscriptable

In [14]:
rfBO = BayesianOptimization(RF,
        {'n_est': (500,1000), 'max_d': (10,40),'max_f':(12,20)})

rfBO.maximize(n_iter=n_iter,**gp_params,acq='ei')

#print('-', * 53)
#print('RF: %f' % rfBO.res['max']['max_val'])


In [ ]:
gbBO = BayesianOptimization(GB,
        {'n_est': (100,500),  #default=100
         'max_d': (3,10),  #default = 3
         'max_f':(12,20)})  #default =None

gbBO.maximize(n_iter=n_iter,**gp_params)


In [19]:
xgBO = BayesianOptimization(XG,
        {'n_est': (100,500), #default=100
         'lr':(.001,.1),  #default=0.1
         'max_d': (3,10), #default=3
         'g':(0,2)})  #default=0

xgBO.explore({'n_est': (100,250,500), 'lr':(.001,.01,.1),'max_d': (5,7,10),'g':(0,2,5)})
xgBO.maximize(n_iter=n_iter,**gp_params)

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |         g |        lr |     max_d |     n_est | 
    1 | 00m01s | -4741.90000 |    0.0000 |    0.0010 |    5.0000 |  100.0000 | 
    2 | 00m07s | -668.20000 |    2.0000 |    0.0100 |    7.0000 |  250.0000 | 
    3 | 00m23s | -558.40000 |    5.0000 |    0.1000 |   10.0000 |  500.0000 | 
    4 | 00m05s | -553.70000 |    0.7771 |    0.0545 |    9.3932 |  119.8669 | 
    5 | 00m11s | -547.20000 |    0.9687 |    0.0174 |    8.2459 |  340.8373 | 
    6 | 00m12s | -554.00000 |    1.4745 |    0.0334 |    6.1033 |  463.1796 | 
    7 | 00m04s | -2367.90000 |    0.7945 |    0.0047 |    6.8583 |  171.5862 | 
    8 | 00m13s | -551.70000 |    1.6009 |    0.0296 |    6.2886 |  446.9461 | 
Bayesian Optimization
-----------------------------------------------------------------------------
 Step |   Time |      Value |         g |        lr |     max_d |     n_est | 
    9 | 00m12s 

In [15]:
# def create_MLP():
#     mlp=models.Sequential()
#     mlp.add(Dense(20, input_dim=X[1].shape[1], activation='relu'))
#     mlp.add(Dense(30, activation='relu'))
#     #mlp.add(Dense(30, activation='relu'))
#     mlp.add(Dense(10, activation='relu'))
#     mlp.add(Dense(1))
#     mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
#     return mlp

def create_MLP(output_size_per_dense_layer):
    
    # Create Sequential and add first layer to take input
    mlp=models.Sequential()
    mlp.add(Dense(output_size_per_dense_layer[0], input_dim=X[1].shape[1], activation='relu'))
    
    # Loop through the rest of the list (except last layer) and add each Dense layer with RELU act func
    for i in range(1,len(output_size_per_dense_layer)-1):
        mlp.add(Dense(output_size_per_dense_layer[i], activation='relu'))
        
    # Add final layer w/o act func, and compile
    mlp.add(Dense(output_size_per_dense_layer[-1]))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    
    return mlp

[20, 30, 10, 1]

In [27]:

def MLP(epochs,batch_size, output_size_per_dense_layer):
    """
    epochs: number of epochs
    batch_size: size of the batch
    output_size_per_dense_layer: sets the output size of each Dense layer in the MLP
        i.e. the default ([20,30,10,1]) will give MLP structure as:
            Dense(20, input_dim=X[1].shape[1], activation='relu')
            Dense(30, activation='relu')
            Dense(30, activation='relu')
            Dense(10, activation='relu')
            Dense(1)
    """
    layers=[ [20,10,10,1],[20,30,10,1],[20,20,10,1] ]

    MLP = KerasRegressor(build_fn=create_MLP, 
                         epochs=int(epochs), 
                         batch_size=int(batch_size), 
                         
                         output_size_per_dense_layer=  layers[int(output_size_per_dense_layer)],
                         verbose=0)    
    mlp_models=[]
    for sku in list(df_dict.keys()):
        mlp_models.append(cross_val_score(MLP,X[sku],y[sku],cv=cv).mean().round(0)  )
    return np.array(mlp_models).mean()

In [36]:
MLP(1,100,[20,30,10,1])

-5235.9

In [29]:
mlpBO = BayesianOptimization(MLP, {
    'epochs': (1,2),
    'batch_size':(100,200),
    'output_size_per_dense_layer':(0,1,2)})
#     'batch_size':(100,200), 
#     'output_size_per_dense_layer':tuple([[20,30,10,1],[20,30,10,1]])})

mlpBO.maximize(n_iter=n_iter,**gp_params)

Initialization
----------------------------------------------------------------------------------------
 Step |   Time |      Value |   batch_size |    epochs |   output_size_per_dense_layer | 
    1 | 00m59s | -5235.90000 |     133.5945 |    1.2530 |                        1.4455 | 


KeyboardInterrupt: 

In [55]:
#TODO Rename variables to avoid confusion

# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# X=np.array([[1,2]]).T
# y=np.array([[1,2]]).T

# # Instanciate a Gaussian Process model
# kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
# gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

# gp.fit(X,y)
# Xpred = np.array([[1,2,3,4,5,6,7,8]]).T

# gp.predict(Xpred, return_std=False)

array([[1.00000000e+00],
       [2.00000000e+00],
       [1.98400072e+00],
       [1.14163840e+00],
       [4.00480845e-01],
       [8.75288404e-02],
       [1.20521832e-02],
       [1.05200283e-03]])

In [71]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae.loc['mean'] = mae.mean()
mae['range']=mae.max(axis=1)-mae.min(axis=1)
mae

,MLP,RF,GB,XG,Best,range
1,419.0,360.0,380.0,361.0,RF,59.0
2,453.0,384.0,410.0,386.0,RF,69.0
3,737.0,584.0,624.0,597.0,RF,153.0
4,791.0,747.0,801.0,793.0,RF,54.0
5,416.0,369.0,386.0,384.0,RF,47.0
6,651.0,552.0,574.0,601.0,RF,99.0
7,762.0,726.0,751.0,742.0,RF,36.0
8,646.0,485.0,485.0,506.0,RF,161.0
9,586.0,655.0,698.0,727.0,MLP,141.0
10,442.0,385.0,411.0,401.0,RF,57.0


In [72]:
#Need lots of rank and counts of how many times models came in 1st  vs 2nd.
def rank_model(df,rank):
    coln = 'Best' + str(rank) 
    sortID = np.argpartition(df[['MLP','RF','GB','XG']].values,rank,axis=1)[:,rank]
    df[coln] = df.columns[sortID]
    

In [73]:
rank_model(mae,0)
rank_model(mae,1)
mae

,MLP,RF,GB,XG,Best,range,Best0,Best1
1,419.0,360.0,380.0,361.0,RF,59.0,RF,XG
2,453.0,384.0,410.0,386.0,RF,69.0,RF,XG
3,737.0,584.0,624.0,597.0,RF,153.0,RF,XG
4,791.0,747.0,801.0,793.0,RF,54.0,RF,MLP
5,416.0,369.0,386.0,384.0,RF,47.0,RF,XG
6,651.0,552.0,574.0,601.0,RF,99.0,RF,GB
7,762.0,726.0,751.0,742.0,RF,36.0,RF,XG
8,646.0,485.0,485.0,506.0,RF,161.0,RF,GB
9,586.0,655.0,698.0,727.0,MLP,141.0,MLP,RF
10,442.0,385.0,411.0,401.0,RF,57.0,RF,XG


In [74]:
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return pred

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [75]:
#NOTE, because we are retraining models here, a bad initiatlization could lead to different results. 

def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":     
            d[sku]=RF_final(X[sku],y[sku])
                    
        elif scores.loc[sku]["Best"]=="MLP":
            d[sku]=MLP_final(X[sku],y[sku])                 
            
        elif scores.loc[sku]["Best"]=="GB":
            d[sku]=GB_final(X[sku],y[sku])
            
        elif scores.loc[sku]["Best"]=="XG":
            d[sku]=XG_final(X[sku],y[sku])
      
    return d     

In [76]:
bm_dict=best_model(mae,X,y)

'{1:               RF\n0    5717.818209\n1    5510.298604\n2    5079.047790\n3    4855.066339\n4    4848.817431\n5       0.000000\n6    4364.450817\n7    4031.301953\n8    3944.842050\n9    3894.189293\n10   4245.188187\n11   4983.633181\n12      0.000000\n13   5744.192988\n14   5223.222718\n15   5362.314867\n16   5163.339575\n17   5257.620845\n18   5379.716069\n19      0.000000\n20   4208.318142\n21   3888.350691\n22   4108.486962\n23   4515.580422\n24   5273.183988\n25   5711.538596\n26      0.000000\n27   6906.761140\n28   6097.723153\n29   5936.196322\n..           ...\n905  5213.389228\n906  5014.941636\n907  4685.789839\n908     0.000000\n909  4154.652278\n910  3857.897641\n911  3824.348997\n912  3772.552890\n913  3924.595310\n914  4097.525466\n915     0.000000\n916  5324.903650\n917  4992.422573\n918  4899.171080\n919  4479.264016\n920  4586.322775\n921  4446.348342\n922     0.000000\n923  3906.095211\n924  3786.133486\n925  3776.057640\n926  3701.543351\n927  3782.033448\n928  

In [77]:
#MAE of Best Model forecasting over all 935 days.  NN do very poorly here!
mae2=[]
for sku in list(df_dict.keys()):
    mae2.append( mean_absolute_error(y[sku], bm_dict[sku])   )
mae2=pd.DataFrame(mae2)
mae2.index = mae2.index + 1
mae2.round(1)

,0
1,191.9
2,215.9
3,304.6
4,361.3
5,197.2
6,189.9
7,352.6
8,201.5
9,392.5
10,198.8


In [78]:
sku_pred=pd.concat([ bm_dict[1],bm_dict[2],bm_dict[3],bm_dict[4],bm_dict[5],bm_dict[6],bm_dict[7],bm_dict[8],bm_dict[9],bm_dict[10],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')

#Each column in this dataframe is the result of 1 of the 4 models
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,5717.818209,6301.653071,9865.725643,10153.291251,5602.367520,7176.216977,10341.561739,6505.311482,6279.433594,5714.657222,73658.036707
1,5510.298604,6512.066582,8556.524297,9330.420316,5765.830248,6921.244257,8090.852354,5673.728950,5267.011719,5539.029505,67167.006832
2,5079.047790,5460.737801,7735.243498,9324.782816,4944.436392,6314.656802,8066.825115,6234.547054,5368.332520,5616.178584,64144.788373
3,4855.066339,4907.510546,8600.125698,9873.201129,5069.712080,6927.614117,8099.700803,5206.362083,5712.226562,5606.811352,64858.330710
4,4848.817431,2609.258951,4299.631053,9935.107310,1835.048437,3765.943200,4538.281949,2510.754229,4950.395996,4541.323872,43834.562428
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.028284,0.000000,-0.028284
6,4364.450817,4004.262165,5161.439033,7940.504973,3824.709259,5972.879755,6844.887051,4286.980709,4488.551758,4817.428118,51706.093638
7,4031.301953,3947.826898,5158.895142,7462.205307,3727.959016,5616.944218,6807.330886,4192.341320,4160.208984,4569.376346,49674.390071
8,3944.842050,4649.660839,5124.796593,7195.689326,3997.489338,5247.771443,6372.850673,3895.344289,3787.561279,4551.669918,48767.675748
9,3894.189293,3881.791308,5029.780984,7370.564596,3713.124132,5554.787497,6780.353349,4827.428066,4408.336914,4520.857199,49981.213338
